###  Imports, Configurations

In [1]:
%%capture
%load_ext rpy2.ipython

In [2]:
%%capture
%reload_ext rpy2.ipython

In [3]:
import ecf
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix

In [4]:
%%capture
%%R
install.packages("dplyr")


In [5]:
%%capture
%%R
library('dplyr')

### Preprocessing

In [6]:
%%R
# Data processing in line with the original ProPublica analysis.
# Same code piece as in the original.
raw_data <- read.csv("./scores-two-years-violent.csv")
df <- dplyr::select(raw_data, age, c_charge_degree, race, age_cat, v_score_text, sex, priors_count, 
                    days_b_screening_arrest, v_decile_score, is_recid, two_year_recid) %>% 
        filter(days_b_screening_arrest <= 30) %>%
        filter(days_b_screening_arrest >= -30) %>% 
        filter(is_recid != -1) %>%
        filter(c_charge_degree != "O") %>%
        filter(v_score_text != 'N/A')


# Write preprocessed data        
write.csv(df, 'violent.csv')

### Descriptive Statistics

In [7]:
compasdata = pd.read_csv('violent.csv').loc[:, ['race', 'sex', 'v_score_text', 'two_year_recid']]
compasdata['v_score_text'] = np.where(compasdata['v_score_text'] == 'Low', 0, 1)
compasdata.columns = ['race', 'sex', 'recid_pred', 'recid_gt']
compasdata['intersection'] = compasdata["sex"] + ' - ' + compasdata["race"]
compasdata = compasdata.astype('string')

In [8]:
compasdata.head()

,race,sex,recid_pred,recid_gt,intersection
0,Other,Male,0,0,Male - Other
1,African-American,Male,0,1,Male - African-American
2,Other,Male,0,0,Male - Other
3,Other,Male,0,0,Male - Other
4,Caucasian,Female,0,0,Female - Caucasian


In [9]:
print('Table III')

table_3 = pd.DataFrame(compasdata.groupby(['race', 'sex']).size()).reset_index().pivot(index='race', columns='sex', values=0).fillna(0)

grand_total = table_3.sum().sum()

table_3['Total'] = table_3.sum(axis=1)
table_3.loc['Total'] = table_3.sum(axis=0)

table_3['%'] = table_3['Total'] / grand_total * 100
table_3.loc['%'] = table_3.sum(axis=0) / 2 / grand_total * 100
table_3.loc['%', '%'] = 0
table_3.round().astype(int)



Table III


sex,Female,Male,Total,%
race,,,,
African-American,393,1525,1918,48
Asian,1,25,26,1
Caucasian,336,1123,1459,36
Hispanic,61,294,355,9
Native American,0,7,7,0
Other,50,205,255,6
Total,841,3179,4020,100
%,21,79,100,0


In [10]:
print('Table IV')

table_4 = pd.DataFrame(confusion_matrix(compasdata['recid_gt'], compasdata['recid_pred']), index = ['-', '+'], columns = ['-', '+'])
table_4.columns.name, table_4.index.name = 'Predicted', 'Actual'
table_4 = table_4.loc[['+', '-'], ['+', '-']].transpose()


grand_total = table_4.sum().sum()

table_4['Total'] = table_4.sum(axis=1)
table_4.loc['Total'] = table_4.sum(axis=0)

table_4['%'] = table_4['Total'] / grand_total * 100
table_4.loc['%'] = table_4.sum(axis=0) / 2 / grand_total * 100
table_4.loc['%', '%'] = 0
table_4.round().astype(int)


Table IV


Actual,+,-,Total,%
Predicted,,,,
+,346,761,1107,28
-,306,2607,2913,72
Total,652,3368,4020,100
%,16,84,100,0


## Fairness Analysis

### 1) Sex

In [11]:
result = ecf.ecf(sampledata=compasdata, s = 'sex', pred = 'recid_pred', gt = 'recid_gt')

In [12]:
print('TABLE V')
print('\n************\n')

print('Table V - Observed (Columns O)')
display(result['contingency'].sort_index().reindex(sorted(result['contingency'].columns, reverse=True), axis=1))

print('\n************\n')

print('Table V - Expected (Columns E)')
display(np.round(result['equal_confusion_test']['expected']).sort_index().reindex(sorted(result['equal_confusion_test']['expected'].columns, reverse=True), axis=1).style.format('{0:,.0f}'))

print('\n************\n')

print("Table V - Residuals (Columns R)")
display(result['posthoc_analysis'].sort_index().reindex(sorted(result['posthoc_analysis'].columns, reverse=True), axis=1).round(1))

TABLE V

************

Table V - Observed (Columns O)


Actual       1         0      
Predicted    1    0    1     0
Female      27   50  137   627
Male       319  256  624  1980


************

Table V - Expected (Columns E)



************

Table V - Residuals (Columns R)


Actual       1         0     
Predicted    1    0    1    0
Female    -6.3 -2.0 -2.2  6.6
Male       6.3  2.0  2.2 -6.6

In [13]:
print('Sex: Equal confusion test p-value is', round(result['equal_confusion_test']['p'], 5))

print('\n************\n')

print('Sex: Confusion parity error is', round(result['confusion_parity_error'], 3))


Sex: Equal confusion test p-value is 0.0

************

Sex: Confusion parity error is 0.121


In [14]:
print('TABLE VII')
print('\n************\n')


print("Table VII - Top Panel - Proportion of Row Totals")
display((result['ratio_all'][1]*100).sort_index().sort_index(axis=1, ascending=False).style.format('{0:,.0f}%'))

print('\n************\n')

print("Table VII - Top Panel - Proportion of Subtotals")
display((result['ratio_pred']*100).sort_index().sort_index(axis=1, ascending=False).style.format('({0:,.0f}%)'))

print('\n************\n')

print("Table VII - Bottom Panel - Proportion of Row Totals")
display((result['ratio_all'][0]*100).sort_index().sort_index(axis=1, ascending=False).style.format('{0:,.0f}%'))

print('\n************\n')

print("Table VII - Bottom Panel - Proportion of Subtotals")
display((result['ratio_gt']*100).sort_index().sort_index(axis=1, ascending=False).style.format('({0:,.0f}%)'))

TABLE VII

************

Table VII - Top Panel - Proportion of Row Totals



************

Table VII - Top Panel - Proportion of Subtotals



************

Table VII - Bottom Panel - Proportion of Row Totals



************

Table VII - Bottom Panel - Proportion of Subtotals


### 2) Race

In [15]:
result = ecf.ecf(sampledata=compasdata, s = 'race', pred = 'recid_pred', gt = 'recid_gt')

In [16]:
print('TABLE VI')
print('\n************\n')

print('Table VI - Observed (Columns O)')
display(result['contingency'].sort_index().reindex(sorted(result['contingency'].columns, reverse=True), axis=1))

print('\n************\n')

print('Table VI - Expected (Columns E)')
display(np.round(result['equal_confusion_test']['expected']).sort_index().reindex(sorted(result['equal_confusion_test']['expected'].columns, reverse=True), axis=1).style.format('{0:,.0f}'))

print('\n************\n')

print("Table VI - Residuals (Columns R)")
display(result['posthoc_analysis'].sort_index().reindex(sorted(result['posthoc_analysis'].columns, reverse=True), axis=1).round(1))

TABLE VI

************

Table VI - Observed (Columns O)


Actual              1         0      
Predicted           1    0    1     0
African-American  250  154  468  1046
Asian               3    0    1    22
Caucasian          64  110  198  1087
Hispanic           10   25   61   259
Native American     1    0    1     5
Other              18   17   32   188


************

Table VI - Expected (Columns E)



************

Table VI - Residuals (Columns R)


Actual              1         0      
Predicted           1    0    1     0
African-American  9.6  1.0  8.5 -13.1
Asian             0.5 -1.5 -2.0   2.1
Caucasian        -7.2 -0.1 -6.5   9.7
Hispanic         -4.1 -0.4 -0.9   3.4
Native American   0.5 -0.8 -0.3   0.4
Other            -0.9 -0.6 -2.7   3.1

In [17]:
print('Race: Equal confusion test p-value is', round(result['equal_confusion_test']['p'], 5))

print('\n************\n')

print('Race: Confusion parity error is', round(result['confusion_parity_error'], 3))


Race: Equal confusion test p-value is 0.0

************

Race: Confusion parity error is 0.133


In [18]:
print('TABLE VIII')
print('\n************\n')


print("Table VIII - Top Panel - Proportion of Row Totals")
display((result['ratio_all'][1]*100).sort_index().sort_index(axis=1, ascending=False).style.format('{0:,.0f}%'))

print('\n************\n')

print("Table VIII - Top Panel - Proportion of Subtotals")
display((result['ratio_pred']*100).sort_index().sort_index(axis=1, ascending=False).style.format('({0:,.0f}%)'))

print('\n************\n')

print("Table VIII - Bottom Panel - Proportion of Row Totals")
display((result['ratio_all'][0]*100).sort_index().sort_index(axis=1, ascending=False).style.format('{0:,.0f}%'))

print('\n************\n')

print("Table VIII - Bottom Panel - Proportion of Subtotals")
display((result['ratio_gt']*100).sort_index().sort_index(axis=1, ascending=False).style.format('({0:,.0f}%)'))

TABLE VIII

************

Table VIII - Top Panel - Proportion of Row Totals



************

Table VIII - Top Panel - Proportion of Subtotals



************

Table VIII - Bottom Panel - Proportion of Row Totals



************

Table VIII - Bottom Panel - Proportion of Subtotals


### 3) Intersectional Groups

In [19]:
result = ecf.ecf(sampledata=compasdata, s = 'intersection', pred = 'recid_pred', gt = 'recid_gt')

In [20]:
print('TABLE IX')
print('\n************\n')

print('Table IX - Observed (Columns O)')
display(result['contingency'].sort_index().reindex(sorted(result['contingency'].columns, reverse=True), axis=1))

print('\n************\n')

print('Table IX - Expected (Columns E)')
display(np.round(result['equal_confusion_test']['expected']).sort_index().reindex(sorted(result['equal_confusion_test']['expected'].columns, reverse=True), axis=1).style.format('{0:,.0f}'))

print('\n************\n')

print("Table IX - Residuals (Columns R)")
display(result['posthoc_analysis'].sort_index().reindex(sorted(result['posthoc_analysis'].columns, reverse=True), axis=1).round(1))

TABLE IX

************

Table IX - Observed (Columns O)


Actual                       1         0     
Predicted                    1    0    1    0
Female - African-American   19   28   81  265
Female - Asian               0    0    0    1
Female - Caucasian           8   15   41  272
Female - Hispanic            0    5    5   51
Female - Other               0    2   10   38
Male - African-American    231  126  387  781
Male - Asian                 3    0    1   21
Male - Caucasian            56   95  157  815
Male - Hispanic             10   20   56  208
Male - Native American       1    0    1    5
Male - Other                18   15   22  150


************

Table IX - Expected (Columns E)



************

Table IX - Residuals (Columns R)


Actual                        1         0      
Predicted                     1    0    1     0
Female - African-American  -2.8 -0.4  0.9   1.1
Female - Asian             -0.3 -0.3 -0.5   0.7
Female - Caucasian         -4.3 -2.3 -3.3   6.5
Female - Hispanic          -2.4  0.2 -2.2   3.1
Female - Other             -2.2 -1.0  0.2   1.7
Male - African-American    11.6  1.2  8.2 -14.2
Male - Asian                0.6 -1.4 -1.9   2.0
Male - Caucasian           -5.1  1.3 -5.0   6.4
Male - Hispanic            -3.3 -0.5  0.1   2.2
Male - Native American      0.5 -0.8 -0.3   0.4
Male - Other                0.1 -0.2 -3.1   2.6

In [21]:
print('Intersectional Groups: Equal confusion test p-value is', round(result['equal_confusion_test']['p'], 5))

print('\n************\n')

print('Intersectional Groups: Confusion parity error is', round(result['confusion_parity_error'], 3))

Intersectional Groups: Equal confusion test p-value is 0.0

************

Intersectional Groups: Confusion parity error is 0.156


In [22]:
print('TABLE X')
print('\n************\n')


print("Table X - Top Panel - Proportion of Row Totals")
display((result['ratio_all'][1]*100).sort_index().sort_index(axis=1, ascending=False).style.format('{0:,.0f}%'))

print('\n************\n')

print("Table X - Top Panel - Proportion of Subtotals")
display((result['ratio_pred']*100).sort_index().sort_index(axis=1, ascending=False).style.format('({0:,.0f}%)'))

print('\n************\n')

print("Table X - Bottom Panel - Proportion of Row Totals")
display((result['ratio_all'][0]*100).sort_index().sort_index(axis=1, ascending=False).style.format('{0:,.0f}%'))

print('\n************\n')

print("Table X - Bottom Panel - Proportion of Subtotals")
display((result['ratio_gt']*100).sort_index().sort_index(axis=1, ascending=False).style.format('({0:,.0f}%)'))

TABLE X

************

Table X - Top Panel - Proportion of Row Totals



************

Table X - Top Panel - Proportion of Subtotals



************

Table X - Bottom Panel - Proportion of Row Totals



************

Table X - Bottom Panel - Proportion of Subtotals
